In [1]:
import math
import torch
import gpytorch
import numpy as np
import numpy.linalg as linalg

import spectralgp

from spectralgp.samplers import AlternatingSampler
from spectralgp.models import ExactGPModel, SpectralModel, ProductKernelSpectralModel

from spectralgp.sampling_factories import ss_factory, ess_factory
from custom_plotting import plot_kernel

import data

import utils
import argparse

import sys
import matplotlib.pyplot as plt

import traceback

torch.set_default_dtype(torch.float64)

In [2]:
train_x, train_y, test_x, test_y, y_std, y_std_train, gen_kern = data.read_data('servo', nx=None, gen_pars=None,
                                                            linear_pars=None,
                                                            spacing='random',
                                                            noise=None)
in_dims = 1 if train_x.dim() == 1 else train_x.size(1)

use_cuda = torch.cuda.is_available()
print('Cuda is available', use_cuda)
if use_cuda:
    torch.set_default_tensor_type(torch.cuda.DoubleTensor)
    train_x, train_y, test_x, test_y, y_std = train_x.cuda(), train_y.cuda(), test_x.cuda(), test_y.cuda(), y_std.cuda()
    if gen_kern is not None:
        gen_kern = gen_kern.cuda()

###########################################
## set up the spectral and latent models ##
###########################################
print("Input Dimensions {}".format(in_dims))



mlatent = 'shared'
shared = True if mlatent == 'shared' else False

Cuda is available True
Input Dimensions 4


In [3]:
data_lh = gpytorch.likelihoods.GaussianLikelihood(noise_prior=gpytorch.priors.SmoothedBoxPrior(1e-8, 1e-3))
data_mod = spectralgp.models.ProductKernelSpectralModel(train_x, train_y, data_lh, shared=shared,
        normalize = False, symmetrize = False, num_locs = 100, spacing='random', pretrain=False, omega_max = 8., nonstat = True)

0
tensor(8.)
0
tensor(8.)
0
tensor(8.)


In [ ]:
alt_sampler = spectralgp.samplers.AlternatingSampler(
    [data_mod], [data_lh], 
    spectralgp.sampling_factories.ss_factory, [spectralgp.sampling_factories.ess_factory],
    totalSamples=10, numInnerSamples=10, numOuterSamples=5, num_dims=in_dims
    )


alt_sampler.run()

Step:  0 Dimension:  0
Loss is:  tensor(-84.5694, grad_fn=<DivBackward0>)
Loss is:  tensor(-83.4238, grad_fn=<DivBackward0>)
Loss is:  tensor(-82.2715, grad_fn=<DivBackward0>)
Loss is:  tensor(-81.1713, grad_fn=<DivBackward0>)
Loss is:  tensor(-80.0580, grad_fn=<DivBackward0>)
Task: 0 ; Iteration 0
Step:  0 Dimension:  1
Loss is:  tensor(-79.2595, grad_fn=<DivBackward0>)
Loss is:  tensor(-78.1643, grad_fn=<DivBackward0>)
Loss is:  tensor(-77.0839, grad_fn=<DivBackward0>)
Loss is:  tensor(-76.0286, grad_fn=<DivBackward0>)
Loss is:  tensor(-75.0021, grad_fn=<DivBackward0>)
Task: 0 ; Iteration 0
Step:  0 Dimension:  2
Loss is:  tensor(-74.0264, grad_fn=<DivBackward0>)
Loss is:  tensor(-73.0022, grad_fn=<DivBackward0>)
Loss is:  tensor(-71.9923, grad_fn=<DivBackward0>)
Loss is:  tensor(-71.0003, grad_fn=<DivBackward0>)
Loss is:  tensor(-70.0197, grad_fn=<DivBackward0>)
Task: 0 ; Iteration 0
Step:  0 Dimension:  3
Loss is:  tensor(-69.0084, grad_fn=<DivBackward0>)
Loss is:  tensor(-68.0426,

In [ ]:
plot_kernel(alt_sampler, data_mod, 'servo')